# Task 5 - Model Training and Tracking

In [1]:
# Install required packages
import pandas as pd
import mlflow
import matplotlib.pyplot as plt

In [ ]:
# Add the parent directory to the system path
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from src.train import ModelTrainer

In [ ]:

# Load transformed data
data = pd.read_csv('F:/Credit-Risk-Model/data/processed/transformed_data_for_training.csv')


In [ ]:
 Initialize ModelTrainer
trainer = ModelTrainer()

# Split data
X_train, X_test, y_train, y_test = trainer.split_data(data, target_col='is_high_risk')

# Train and tune models
results = trainer.train_and_tune(X_train, y_train, X_test, y_test, experiment_name="CreditRiskExperiment")


In [ ]:
# Visualize results
metrics_df = pd.DataFrame({
    model_name: result['metrics'] for model_name, result in results.items()
}).T
print("Model Performance Metrics:")
print(metrics_df)

In [ ]:

# Plot metrics
metrics_df.plot(kind='bar', figsize=(10, 6))
plt.title('Model Performance Comparison')
plt.ylabel('Score')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('F:/Credit-Risk-Model/results/model_performance.png')
plt.show()



In [ ]:
# Register best model
if trainer.best_model:
    with mlflow.start_run(run_name=f"Best_Model_{trainer.best_model_name}"):
        mlflow.sklearn.log_model(trainer.best_model, trainer.best_model_name)
        run_id = mlflow.active_run().info.run_id
        trainer.register_best_model(trainer.best_model_name, trainer.best_model, run_id)

In [ ]:
 Load transformed data
    data = pd.read_csv(
        'F:/Credit-Risk-Model/data/processed/transformed_data_for_training.csv')

    # Initialize trainer
    trainer = ModelTrainer()

    # Split data
    X_train, X_test, y_train, y_test = trainer.split_data(
        data, target_col='is_high_risk')

    # Train and tune models
    results = trainer.train_and_tune(X_train, y_train, X_test, y_test)

    # Register best model
    if trainer.best_model:
        with mlflow.start_run(run_name=f"Best_Model_{
            trainer.best_model_name}"
        ):
            mlflow.sklearn.log_model(
                trainer.best_model, trainer.best_model_name)
            run_id = mlflow.active_run().info.run_id
            trainer.register_best_model(
                trainer.best_model_name, trainer.best_model, run_id)
